In [2]:
%load_ext autoreload
%autoreload 2

from filemanager import fileset
from cli import cli

fileset_manager = fileset.PDSFileset()

qgis_logs = r'Y:\PDS-Projects\DUNES'
filesets_folder = r'Y:\PDS-Projects\DUNES'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# pick qgis log
qgis_log_list = cli.search(qgis_logs, '.csv')

Searching *.csv files in directory:Y:\PDS-Projects\DUNES
Scanned filesets are: 

0 Y:\PDS-Projects\DUNES\fs_Dune_02_ABP47_Ryb.csv
1 Y:\PDS-Projects\DUNES\fs_Dune_02_ABP48_Ryb.csv


In [15]:
qgis_log_picked = cli.pick_from_list(qgis_log_list)

Please type two numbers that define the range of filesets you like to copy or move
Please type the first number: 
Please type the second number: 
Do you want to add more elements? (Y/N)


Picked elements:
fs_Dune_02_ABP48_Ryb.csv


In [16]:
qgis_log_picked

['Y:\\PDS-Projects\\DUNES\\fs_Dune_02_ABP48_Ryb.csv']

In [19]:
import shutil
import os

def qgislogToFilesets(output_dir, qgis_log, has_svp = False):
    
    filesets = []
    svp_filesets = []
    filesets_dict = {}

    with open(qgis_log, 'r') as csv_file:
        file_content = csv_file.read().splitlines()

        header = file_content[0].split(',')
        
        filename_idx = 0
        svp_idx = 0
        fileset_idx = 0
        print(header)
        print(filename_idx)
        for field in header:
            if field == 'Fname' or field == 'fname' or field == 'FileName':
                break
            else:
                filename_idx += 1
        if has_svp:
            for field in header:
                if field == 'svp_station':
                    break
                else:
                    svp_idx += 1
                
        for field in header:
            if field == 'fileset':
                break
            elif field == 'FilesetTmp':
                break
            else:
                fileset_idx += 1

    
        for num, line in enumerate(file_content[1:]):
            line_content = line.split(',')
            file_name = line_content[filename_idx]
            fileset_name = line_content[fileset_idx]
            if has_svp:
                svp_name = line_content[svp_idx]
            else:
                svp_name = 'None'

            if fileset_name in filesets_dict.keys():
                filesets_dict[fileset_name].append((file_name, svp_name))
            else:
                filesets_dict[fileset_name] = []
                filesets_dict[fileset_name].append((file_name, svp_name))
    
    for key in filesets_dict.keys():
        fileset_name = key

        fs = fileset.PDSFileset()
        fs.setname(fileset_name)
        
        for filename, svp_name in filesets_dict[key]:
            fs.addfile(filename=filename, svp_name=svp_name)
        filesets.append(fs)
        svp_fs = fs.create_svp_filesets()
        svp_filesets.append(svp_fs)
    
    
    for fs, svpfs in zip(filesets,svp_filesets):
        fs_dir = os.path.join(output_dir,fs.name)
        
        if not os.path.exists(fs_dir):
            os.mkdir(fs_dir)
            
        print(f'{fs.name} - {len(fs.files)}')
        fs.savefileset(dirpath=fs_dir)
        len_svp = 0
        for svfs in svpfs:
            print(f'{svfs.name} - {len(svfs.files)}')
            svfs.savefileset(dirpath=fs_dir)
            len_svp += len(svfs.files)
        print(len_svp)

In [20]:
qgislogToFilesets(filesets_folder, qgis_log_picked[0])

['FileName', 'begin', 'end', 'Project', 'Fileset', 'ProcessedB', 'Status', 'Date', 'Time', 'Projection', 'FilesetTmp', 'Fileset_BS']
0
fs_Dunes_02_ABP48_Ryb - 154
fs_Dunes_02_ABP48_Ryb_None - 154
154
